In [6]:
import os
import cv2
import numpy as np
from pprint import pprint
import json

### Util functions

In [7]:
import matplotlib.pyplot as plt

def show_cv_img(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [8]:
def show_corners(img, corners):
    for corner in corners:
        cv2.circle(img, tuple(map(int, corner)), radius=10, color=(255, 0, 0), thickness=-1)

### Background

In [9]:
DATA_DIR = 'data/'
DIR_SRC = DATA_DIR + 'background/'
DIR_DST = DATA_DIR + 'synthetic/resized_bg/'
if not os.path.exists(DIR_DST):
    os.mkdir(DIR_DST)

In [11]:
for i, bg in enumerate(os.listdir(DIR_SRC)):
    img = cv2.imread(DIR_SRC + bg)
    resized = cv2.resize(img, (1024, 680))
    
    print(f"{bg} : {img.shape}")
    resized_path = DIR_DST + 'bg_' + str(i) + '.png'
    if not os.path.exists(resized_path):
        cv2.imwrite(resized_path, resized)

1000_F_298240692_uohz4LEqlbe0lMlRhPOBzXcNVFyl8Wws.jpg : (667, 1000, 3)
1557845371_sangach_(8).jpg : (768, 1024, 3)
2-56a2fc1b5f9b58b7d0cffb23.jpg : (600, 900, 3)
3703784863_1bfb1c0197.jpg : (375, 500, 3)
41532873-pink-fabric-texture-textile-background-woolen-texture-background-knitted-wool-fabric-pink-hairy-fluf.webp : (866, 1300, 3)
6916821778_86b64ecc04_b.jpg : (576, 1024, 3)
88f834fd8f6641bf4056a128f654fc75.jpg : (934, 1400, 3)
background-1838494__480.jpg : (480, 719, 3)
bg_0.jpeg : (1024, 1365, 3)
bg_1.png : (1024, 1365, 3)
bg_2.jpg : (1024, 1024, 3)
bg_3.jpg : (523, 640, 3)
bg_4.jpg : (631, 950, 3)
bg_5.png : (376, 550, 3)
bg_6.jpg : (1280, 1440, 3)
bg_7.png : (866, 1300, 3)
bg_8.png : (866, 1300, 3)
bg_9.jpg : (1280, 1280, 3)
brown-fabric-wallpaper-texture-background-vintage-73335917.jpg : (533, 800, 3)
carpet-surface-closeup-as-background-36601006.jpg : (533, 800, 3)
depositphotos_128693244-stock-photo-closeup-surface-abstract-fabric-pattern.jpg : (768, 1024, 3)
depositphotos_34

### v1.0

#### Copy imgs

In [11]:
DATA_DIR = 'data/'
DIR_ORG = DATA_DIR + 'v1.0.2/'
DIR_SYN = DATA_DIR + 'synthetic/origin/'

In [12]:
for dir in os.listdir(DIR_SYN):
    print(f"{dir} : {len(os.listdir(DIR_SYN + dir))}")

CitizenCardV1_back : 385
CitizenCardV1_front : 400
CitizenCardV2_back : 340
CitizenCardV2_front : 393
IdentificationCard_back : 384
IdentificationCard_front : 383
LicenseCard : 189
Other : 36
Passport : 10


In [30]:
dirs = os.listdir(DIR_ORG)
dirs

['cccd_v1_back',
 'cccd_v1_front',
 'cccd_v2_back',
 'cccd_v2_front',
 'cmnd_v1_back',
 'cmnd_v1_front']

In [34]:
new_dirs = ['CitizenCardV1_back', 'CitizenCardV1_front', 'CitizenCardV2_back', 'CitizenCardV2_front', 'IdentificationCard_back', 'IdentificationCard_front']

In [40]:
import shutil

for j, dir in enumerate(dirs):
    src_dir = DIR_ORG + dir + '/images/part_1/'
    imgs = list(sorted(os.listdir(src_dir)))

    dst_dir = DIR_SYN + new_dirs[j] + '/'
    os.makedirs(dst_dir, exist_ok=True)
    
    i = 0
    while i < len(imgs):
        img_name = imgs[i].split('.')[0]
        img_name = '_'.join(p for p in img_name.split('_')[1:-1])
        if img_name.startswith('_'):
            img_name = '_'.join(p for p in img_name.split('_')[1:])
        
        if not os.path.exists(dst_dir + imgs[i]):
            shutil.copy(src_dir + imgs[i], dst_dir + img_name + '.png')
        i += 5

    print(f"{dir} : {len(imgs)} ---> {new_dirs[j]} : {len(os.listdir(dst_dir))}")



cccd_v1_back : 1925 ---> CitizenCardV1_back : 385
cccd_v1_front : 2000 ---> CitizenCardV1_front : 400
cccd_v2_back : 1700 ---> CitizenCardV2_back : 340
cccd_v2_front : 1965 ---> CitizenCardV2_front : 393
cmnd_v1_back : 1920 ---> IdentificationCard_back : 384
cmnd_v1_front : 1915 ---> IdentificationCard_front : 383


#### Create original data for `LicenceCard`, `Other`, `Passport` classes

In [69]:
DATA_DIR = 'data/'

DIR_SRC = DATA_DIR + 'clfn/'
DIR_DST = DATA_DIR + 'synthetic/origin/'

In [114]:
anno_dir = DIR_SRC + 'annos/'
img_dir = DIR_SRC + 'images/'


for dir in os.listdir(img_dir):
    # Read anno file
    anno = json.load(open(anno_dir + dir + '.json'))

    if len(anno['images']) != len(anno['annotations']):
        print(f"Different size: {len(anno['images'])} images with {len(anno['annotations'])} annos for [{dir}]")
        continue

    for i in range(len(anno['images'])):
        # Take image file

        img_file = anno['images'][i]['file_name']
        img_path = img_dir + dir + '/' + img_file

        if os.path.exists(img_path):
            # Load image
            img = cv2.imread(img_path)

            # Take coordinates of segmentation points
            coords = anno['annotations'][i]['segmentation'][0]
            points = []
            for i in range(len(coords)):
                if i % 2 == 0:
                    points.append(tuple(map(int, (coords[i], coords[i+1]))))

            # Convert to numpy array
            pts = np.array([np.array(i) for i in points])

            # Get 4 corners
            s = pts.sum(axis=1)
            tl = pts[np.argmin(s)]
            br = pts[np.argmax(s)]

            diff = np.diff(pts, axis=1)
            tr = pts[np.argmin(diff)]
            bl = pts[np.argmax(diff)]
            
            corners = [tl, tr, br, bl]

            # Define source points on original image
            src = np.array(corners, dtype='float32')

            # Get the shape of new image
            mins = np.min(pts, axis=0)
            x_min, y_min = map(int, (mins[0], mins[1]))
            maxs = np.max(pts, axis=0)
            x_max, y_max = map(int, (maxs[0], maxs[1]))

            new_w = x_max - x_min
            new_h = y_max - y_min

            # Define destination points on new image
            dst = np.array([
                [0, 0],
                [new_w - 1, 0],
                [new_w - 1, new_h - 1],
                [0, new_h]],
                dtype='float32')

            # Perform 'reversed' perspective transform
            trans_mat = cv2.getPerspectiveTransform(src, dst)
            warp = cv2.warpPerspective(img, trans_mat, (new_w, new_h))

            dst_dir = DIR_DST + dir + '/'
            if not os.path.exists(dst_dir):
                os.mkdir(dst_dir)
            cv2.imwrite(dst_dir + img_file, warp)

        else:
            print(f"Missing image [{img_file}]")


### v2.0

#### `OCR` dataset

In [9]:
ROOT_DIR = 'data/OCR/'
DST_DIR = 'data/synthetic/origin/v2.0/'

In [ ]:
for dir in os.listdir(ROOT_DIR):
    img_dir = os.path.join(ROOT_DIR, dir, 'img/')
    ann_dir = os.path.join(ROOT_DIR, dir, 'ann/')
    
    for img_file in os.listdir(img_dir):
        img = cv2.imread(img_dir + img_file)
        
        ann_file = img_file + '.json'
        if not os.path.exists(ann_dir + ann_file):
            print(f"No anno file for [{img_file}]")
            continue
        
        anno = json.load(open(ann_dir + ann_file))
        # pprint(anno)
        points = []
        for obj in anno['objects']:
            if obj['classTitle'] == 'square':
                points.append(obj['points']['exterior'])
        if len(points) == 0:
            print(f"Invalid anno for [{img_file}]")
            continue
        
        pts = np.array(points[0])

        # Get 4 corners
        s = pts.sum(axis=1)
        tl = pts[np.argmin(s)]
        br = pts[np.argmax(s)]

        diff = np.diff(pts, axis=1)
        tr = pts[np.argmin(diff)]
        bl = pts[np.argmax(diff)]
        
        corners = [tl, tr, br, bl]

        # Define source points on original image
        src = np.array(corners, dtype='float32')

        # Get the shape of new image
        mins = np.min(pts, axis=0)
        x_min, y_min = map(int, (mins[0], mins[1]))
        maxs = np.max(pts, axis=0)
        x_max, y_max = map(int, (maxs[0], maxs[1]))

        new_w = x_max - x_min
        new_h = y_max - y_min

        # Define destination points on new image
        dst = np.array([
            [0, 0],
            [new_w - 1, 0],
            [new_w - 1, new_h - 1],
            [0, new_h]],
            dtype='float32')

        # Perform 'reversed' perspective transform
        trans_mat = cv2.getPerspectiveTransform(src, dst)
        warp = cv2.warpPerspective(img, trans_mat, (new_w, new_h))
        
        if new_w < new_h:
            warp = cv2.rotate(warp, cv2.ROTATE_90_COUNTERCLOCKWISE)

        dst_dir = os.path.join(DST_DIR, dir)
        if not os.path.exists(dst_dir):
            os.mkdir(dst_dir)
        if not os.path.exists(os.path.join(dst_dir, img_file)):
            cv2.imwrite(os.path.join(dst_dir, img_file), warp)
        else:
            print(f"Duplicate name for [{img_file}]")
        
    #     break
    # break

#### `eKYC_segmentation` dataset

In [25]:
ROOT_DIR = 'data/eKYC_segmentation/'
DST_DIR = 'data/synthetic/origin/v2.0/pool/'

In [20]:
from pycocotools.coco import COCO

In [33]:
for dir in ['train', 'val']:
    img_dir = os.path.join(ROOT_DIR, dir, 'JPEGImages/')
    
    anno_file = f"{dir}_annotations.json"
    coco = COCO(os.path.join(ROOT_DIR, dir, anno_file))
    img_idx = list(coco.imgs.keys())
    for i in img_idx:
        img_file = coco.loadImgs(i)[0]['file_name'].split('/')[1]
        img_name = img_file.split('.')[0]
        if img_name.split('_')[-1] == 'flip':
            continue
        img_path = os.path.join(img_dir, img_file)
        img = cv2.imread(img_path)
        # print(img_file)
        ann_ids = coco.getAnnIds(imgIds=i)
        annos = coco.loadAnns(ann_ids)
        annos = [anno for anno in annos if anno['category_id'] == 0]
        # print(len(annos))
        # pprint(annos[0])
        
        if len(annos) != 1:
            print(f"Invalid anno for [{img_file}]")
            continue
        
        segm = annos[0]['segmentation'][0]
        points = []
        for i in range(len(segm)-1):
            if i % 2 == 0:
                points.append(list(map(int, (segm[i], segm[i+1]))))
        pts = np.array(points)
        
        # Get 4 corners
        s = pts.sum(axis=1)
        tl = pts[np.argmin(s)]
        br = pts[np.argmax(s)]

        diff = np.diff(pts, axis=1)
        tr = pts[np.argmin(diff)]
        bl = pts[np.argmax(diff)]
        
        corners = [tl, tr, br, bl]

        # Define source points on original image
        src = np.array(corners, dtype='float32')

        # Get the shape of new image
        mins = np.min(pts, axis=0)
        x_min, y_min = map(int, (mins[0], mins[1]))
        maxs = np.max(pts, axis=0)
        x_max, y_max = map(int, (maxs[0], maxs[1]))

        new_w = x_max - x_min
        new_h = y_max - y_min

        # Define destination points on new image
        dst = np.array([
            [0, 0],
            [new_w - 1, 0],
            [new_w - 1, new_h - 1],
            [0, new_h]],
            dtype='float32')

        # Perform 'reversed' perspective transform
        trans_mat = cv2.getPerspectiveTransform(src, dst)
        warp = cv2.warpPerspective(img, trans_mat, (new_w, new_h))
        
        if new_w < new_h:
            warp = cv2.rotate(warp, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
        if not os.path.exists(os.path.join(DST_DIR, img_file)):
            cv2.imwrite(os.path.join(DST_DIR, img_file), warp)
        else:
            print(f"Duplicate name for [{img_file}]")
    
    #     break    
    # break

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
Invalid anno for [eKYC_dataset_1112.jpg]
Invalid anno for [FB_IMG_1628999696646.jpg]
Invalid anno for [eKYC_dataset_1495.jpg]
Invalid anno for [eKYC_dataset_983.jpg]
Invalid anno for [FB_IMG_1628997334317.jpg]
Invalid anno for [eKYC_dataset_883.jpg]
Invalid anno for [eKYC_dataset_1018.jpg]
Invalid anno for [FB_IMG_1628999792318.jpg]
Invalid anno for [FB_IMG_1629000130091.jpg]
Invalid anno for [FB_IMG_1628998681123.jpg]
Invalid anno for [eKYC_dataset_1734.jpg]
Invalid anno for [FB_IMG_1628999824486.jpg]
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Invalid anno for [eKYC_dataset_255.jpg]
Invalid anno for [eKYC_dataset_434.jpg]


### Data Synthesis

In [50]:
DATA_DIR = 'data/synthetic/'
DIR_BG = DATA_DIR + 'resized_bg/'
DIR_ORG = DATA_DIR + 'origin/v2.0/'
DIR_SYN = DATA_DIR + 'synthesis/'

# for s in ['images/', 'masks']:
#     if not os.path.exists(DIR_SYN + s):
#         os.mkdir(DIR_SYN + s)

In [13]:
dirs = list(sorted(os.listdir(DIR_ORG)))
for dir in dirs:
    print(f"{dir} : {len(os.listdir(DIR_ORG + dir))}")

CitizenCardV1_back : 49
CitizenCardV1_front : 719
CitizenCardV2_back : 44
CitizenCardV2_front : 33
IdentificationCard_back : 68
IdentificationCard_front : 840
LicenseCard : 170
Other : 36
Passport : 10


In [14]:
def get_params(width: int, height: int, distortion_scale: float):
    """Get parameters for ``perspective`` for a random perspective transform.

    Args:
        width (int): width of the image.
        height (int): height of the image.
        distortion_scale (float): argument to control the degree of distortion and ranges from 0 to 1.

    Returns:
        List containing [top-left, top-right, bottom-right, bottom-left] of the original image,
        List containing [top-left, top-right, bottom-right, bottom-left] of the transformed image.
    """
    half_height = height // 2
    half_width = width // 2
    topleft = np.array([
        int(np.random.randint(0, int(distortion_scale * half_width) + 1, size=(1,))),
        int(np.random.randint(0, int(distortion_scale * half_height) + 1, size=(1,))),
    ])
    topright = np.array([
        int(np.random.randint(width - int(distortion_scale * half_width) - 1, width, size=(1,))),
        int(np.random.randint(0, int(distortion_scale * half_height) + 1, size=(1,))),
    ])
    botright = np.array([
        int(np.random.randint(width - int(distortion_scale * half_width) - 1, width, size=(1,))),
        int(np.random.randint(height - int(distortion_scale * half_height) - 1, height, size=(1,))),
    ])
    botleft = np.array([
        int(np.random.randint(0, int(distortion_scale * half_width) + 1, size=(1,))),
        int(np.random.randint(height - int(distortion_scale * half_height) - 1, height, size=(1,))),
    ])
    startpoints = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype='float32')
    endpoints = np.array([topleft, topright, botright, botleft], dtype='float32')
    return startpoints, endpoints

In [15]:
global_avg_w, global_avg_h = 0, 0

for j, dir in enumerate(dirs):
    avg_w, avg_h = 0, 0
    for i, file in enumerate(os.listdir(DIR_ORG + dir + '/')):
        img = cv2.imread(DIR_ORG + dir + '/' + file)

        h, w = img.shape[:2]

        avg_w += w
        avg_h += h

    global_avg_w += avg_w / len(os.listdir(DIR_ORG + dir + '/'))
    global_avg_h += avg_h / len(os.listdir(DIR_ORG + dir + '/'))

print(global_avg_w / len(dirs))
print(global_avg_h / len(dirs))

714.4121450568117
461.1123405514831


In [73]:
def resize_obj(img):
    h, w = img.shape[:2]

    new_h = int(h * 720 / w)
    if new_h > 480:
        new_w = int(720 * 480 / new_h)
        img = cv2.resize(img, (new_w, 480))
    else:
        img = cv2.resize(img, (720, new_h))
    
    ratio = np.random.choice(np.arange(1.35, 1.4, 0.05))
    h, w = img.shape[:2]
    img = cv2.resize(img, (int(ratio * w), int(ratio * h)))
    # print(w, h)
    return img

In [97]:
def transform(img, width, height):
    # Perspective transform
    src, dst = get_params(width, height, distortion_scale=0.2)
    trans_mat = cv2.getPerspectiveTransform(src, dst)
    warp = cv2.warpPerspective(img, trans_mat, (width, height))

    mins = np.min(dst, axis=0)
    x_min, y_min = map(int, (mins[0], mins[1]))
    maxs = np.max(dst, axis=0)
    x_max, y_max = map(int, (maxs[0], maxs[1]))

    cut = warp[y_min:y_max, x_min:x_max]
    
    diff = np.array([[x_min, y_min]])
    dst_cut = dst - diff

    return cut, dst_cut

In [104]:
def random_rotate(bg, img, dst, p):
    height, width = img.shape[:2]
    # Random rotate
    if p == 1:
        rt_bg = cv2.rotate(bg, cv2.ROTATE_180)
        rt_cut = cv2.rotate(img, cv2.ROTATE_180)
        tl = np.array([width - dst[2][0], height - dst[2][1]])
        tr = np.array([width - dst[3][0], height - dst[3][1]])
        br = np.array([width - dst[0][0], height - dst[0][1]])
        bl = np.array([width - dst[1][0], height - dst[1][1]])
        new_dst = np.array([tl ,tr, br, bl])
        return rt_bg, rt_cut, new_dst
    elif p == 2:
        rt_bg = cv2.rotate(bg, cv2.ROTATE_90_CLOCKWISE)
        rt_cut = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        tl = np.array([height - dst[3][1], dst[3][0]])
        tr = np.array([height - dst[0][1], dst[0][0]])
        br = np.array([height - dst[1][1], dst[1][0]])
        bl = np.array([height - dst[2][1], dst[2][0]])
        new_dst = np.array([tl ,tr, br, bl])
        return rt_bg, rt_cut, new_dst
    elif p == 3:
        rt_bg = cv2.rotate(bg, cv2.ROTATE_90_COUNTERCLOCKWISE)
        rt_cut = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        tl = np.array([dst[1][1], width - dst[1][0]])
        tr = np.array([dst[2][1], width - dst[2][0]])
        br = np.array([dst[3][1], width - dst[3][0]])
        bl = np.array([dst[0][1], width - dst[0][0]])
        new_dst = np.array([tl ,tr, br, bl])
        return rt_bg, rt_cut, new_dst
    else:
        return bg, img, dst

In [18]:
def get_coloured_mask(mask):
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    r[mask == 1], g[mask == 1], b[mask == 1] = [128, 0, 128]
    coloured_mask = np.stack([r, g, b], axis=2)
    return coloured_mask

In [111]:
def augment(img_info, rate=1, show_annos=False):
    # Init lists for annotation
    images, annotations = [], []

    org_img = cv2.imread(img_info['path'])
    img = org_img.copy()

    for r in range(rate):
        ## Resize img with random ratio
        img = resize_obj(img)
        height, width = img.shape[:2]
        
        ## Transform img
        cut, dst_cut = transform(img, width, height)
        print(cut.shape)
        cut_height, cut_width = cut.shape[:2]
        
        ## Get random background
        bg_num = np.random.randint(0, 130)
        bg = cv2.imread(DIR_BG + f'bg_{bg_num}.png')
        
        ## Random rotate
        p = np.random.choice(4, p=[0.25, 0.25, 0.25, 0.25])
        bg, cut, dst_cut = random_rotate(bg, cut, dst_cut, p)
        bg_org = bg.copy()
        bg_height, bg_width = bg.shape[:2]
        if p == 2 or p == 3:
            cut_width, cut_height = cut_height, cut_width
        print(p, cut.shape)
        
        ## Create mask of object
        mask_cut = np.zeros(cut.shape, dtype='uint8')        
        cv2.drawContours(mask_cut, [dst_cut.reshape((-1, 1, 2)).astype('int32')], -1, (255, 255, 255), -1)

        ## Create random root point
        print(bg.shape, img.shape, cut.shape)
        rand_x = np.random.randint(0, bg_width - cut_width - 1)
        rand_y = np.random.randint(0, bg_height - cut_height - 1)
        root_point = (rand_x, rand_y)

        ## Calculate new corners and convert to segmentation for annos
        corners = dst_cut + np.array([[rand_x, rand_y]])

        ## Combine object with random background to create composite image
        fin_cut = cv2.cvtColor(cut, cv2.COLOR_RGB2RGBA)
        blue, green, red, a = cv2.split(fin_cut)
        a = mask_cut
        fin_cut[:, :, 0] = cut[:, :, 0]
        fin_cut[:, :, 1] = cut[:, :, 1]
        fin_cut[:, :, 2] = cut[:, :, 2]
        fin_cut[:, :, 3] = a[:, :, 0]

        alpha_s = fin_cut[:, :, 3] / 255.
        alpha_l = 1. - alpha_s

        for c in range(3):
            bg[rand_y:rand_y+cut_height, rand_x:rand_x+cut_width, c] = (alpha_s * cut[:, :, c] +
                                alpha_l * bg[rand_y:rand_y+cut_height, rand_x:rand_x+cut_width, c])

        ## Smooth edges with Gaussian Blur
        mask_bg = np.zeros(bg.shape, dtype=np.uint8)
        mask_bg[rand_y:rand_y+cut_height, rand_x:rand_x+cut_width] = mask_cut

        alpha = cv2.GaussianBlur(mask_bg, (15, 15), 50).astype('float32') / 255.

        comp = alpha * bg.astype('float32') + (1 - alpha) * bg_org.astype('float32')

        
        ## Show annos
        if show_annos:
            # Bounding box
            cv2.rectangle(comp, (rand_x, rand_y), (rand_x+cut_width, rand_y+cut_height), (0, 255, 0))

            # Corners and segmentation mask
            for corner in corners:
                cv2.circle(comp, tuple(map(int, corner)), radius=5, color=(0, 0, 255), thickness=-1)
            mask = np.zeros((bg_height, bg_width), dtype=np.uint8)
            mask = cv2.fillPoly(mask, np.int32([corners]), 1)
            rgb_mask = get_coloured_mask(mask)
            # print(comp.shape, rgb_mask.shape)
            comp = cv2.addWeighted(comp.astype(np.uint8), 1, rgb_mask, 0.35, 0)

            # Root point
            cv2.circle(comp, root_point, radius=5, color=(255, 0, 0), thickness=-1)
        

        # comp, corners = random_rotate(comp, corners, p)
        segment = [[]]
        for corner in corners:
            segment[0].extend(list(corner))
                
        ## Write composite image & mask image
        if not os.path.exists(img_info['dst_dir'] + 'images/'):
            os.mkdir(img_info['dst_dir'] + 'images/')
        img_name = f"{img_info['class']}_{img_info['num']}_aug_{r}.png"
        cv2.imwrite(img_info['dst_dir'] + 'images/' + img_name, comp)

        # mask_name = f"{img_info['class']}_{img_info['num']}_aug_{r}_mask.png"
        # cv2.imwrite(DIR_SYN + 'masks/' + mask_name, mask_bg)

        ## Write annos
        image = {
            'id': img_info['start_id'] + r,
            'file_name': img_name,
            'height': comp.shape[0],
            'width': comp.shape[1]
            # 'mask_name': mask_name
        }

        annotation = {
            'id': img_info['start_id'] + r,
            'image_id': img_info['start_id'] + r,
            'iscrowd': 0,
            'category_id': img_info['cls_id'],
            'bbox': [rand_x * 1., rand_y * 1., cut_width * 1., cut_height * 1.],
            'area': cut_width * cut_height * 1.,
            'segmentation': segment
        }

        images.append(image)
        annotations.append(annotation)
    return images, annotations

        

In [ ]:
images, annotations = [], []
curr_id = 0
DATA_VER = 'v2.1'

for cls_id, dir in enumerate(dirs):
    src_dir = DIR_ORG + dir + '/'
    dst_dir = DIR_SYN + DATA_VER + '/'
    
    num_img = len(os.listdir(src_dir))
    # print(num_img)
    if num_img > 400:
        rate = 1
    else:
        rate = 400 // num_img

    for img_num, img_file in enumerate(os.listdir(src_dir)):
        img_path = src_dir + img_file
        
        img_info = {
            'start_id': curr_id,
            'num': img_num,
            'path': img_path,
            'class': dir,
            'cls_id': cls_id + 1,
            'dst_dir': dst_dir
        }
        
        images_, annotations_ = augment(img_info, rate=rate, show_annos=False)
        images.extend(images_)
        annotations.extend(annotations_)

        curr_id += rate
        # break
    
    print(f"{dir} : {num_img * rate}")

    # break


In [113]:
with open(DIR_SYN + DATA_VER + '/annotations.json', 'w+') as f:
    json.dump({
        'info': {
            'description': "eKYC Synthetic Segmentation Dataset",
            'version': DATA_VER,
            'date_created': "20/04/2022",
            'contributor': "phatdp, cuonghv, linhlth"
        },
        'categories': [
            {'id': 1, 'name': "CitizenCardV1_back"},
            {'id': 2, 'name': "CitizenCardV1_front"},
            {'id': 3, 'name': "CitizenCardV2_back"},
            {'id': 4, 'name': "CitizenCardV2_front"},
            {'id': 5, 'name': "IdentificationCard_back"},
            {'id': 6, 'name': "IdentificationCard_front"},
            {'id': 7, 'name': "LicenseCard"},
            {'id': 8, 'name': "Other"},
            {'id': 9, 'name': "Passport"}
        ],
        'images': images,
        'annotations': annotations
    }, f, indent=4)